# 具体流程大致如下：
# 1.获取数据
# 2.基本数据处理
#  2.1 缺失值处理
#  2.2 确定特征值，目标值
#  2.3 分割数据
# 3.特征工程(进行标准化处理)
# 4.进行机器学习(逻辑回归)
# 5.模型评估

In [221]:
# 导入必要的包
import pandas as pd
import numpy as ny
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
# export_graphviz 函数是 sklearn.tree 模块中的一个工具，用于将决策树模型导出为 Graphviz 格式的 DOT 文件。
# 这个格式可以被 Graphviz 软件可视化，便于分析和理解决策树的结构。
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [222]:
# 1.获取数据
titanic = pd.read_csv("./data/Titanic.txt",encoding='ISO-8859-1')
titanic.head()

# 检查 Pclass 列
print(titanic['Pclass'].unique())

# 将 Pclass 转为字符串类型（如有必要）
titanic['Pclass'] = titanic['Pclass'].astype(str)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,ParCh,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [223]:
# 2.基本数据处理
titanic = titanic.dropna(subset=["Survived"])
#  2.1 确定特征值，目标值
x = titanic[["Pclass","Age","Sex"]]
y = titanic["Survived"]

#  2.2 缺失值处理
# inplace=True：这个参数表示在原地修改数据框，即直接在 x 数据框中替换缺失值，而不是返回一个新的数据框。
x['Age'].fillna(x['Age'].mean(), inplace=True)

#  2.3 分割数据
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=22)


C:\Users\ZSJ\AppData\Local\Temp\ipykernel_10124\2649128600.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['Age'].fillna(x['Age'].mean(), inplace=True)
C:\Users\ZSJ\AppData\Local\Temp\ipykernel_10124\2649128600.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Age'].fillna(x['Age'].mean(), inplace=True)


In [224]:
# 3.特征工程
# 对于x转换成字典数据x.to_dict(orient="records")
# 这个方法将数据框 x 转换为字典格式，每一行转换为一个字典，字典的键是列名，值是对应的列值。
# 例如，若 x 数据框的一行包含 pclass、age 和 sex 三个特征，则转换后可能得到 {"pclass": "1st", "age": 29.00, "sex": "female"} 这样的字典。
# orient="records" 参数表示每一行转换为一个字典，并将所有这些字典放入一个列表中。
# 例如：[{"pclass": "1st", "age": 29.00, "sex": "female"}, {}]

# DictVectorizer 将字典格式的数据转换为数组格式。
# sparse=False 参数表示输出为一个密集数组（即常规的 NumPy 数组），而不是稀疏矩阵。
transfer = DictVectorizer(sparse=False)

# 在训练集上拟合并转换
x_train = transfer.fit_transform(x_train.to_dict(orient="records"))
# 在测试集上仅转换
x_test = transfer.transform(x_test.to_dict(orient="records"))

print(f"训练集特征数量: {x_train.shape[1]}")
print(f"测试集特征数量: {x_test.shape[1]}")

# 获取特征名称
feature_names = transfer.get_feature_names_out()
print(f"特征名称: {feature_names}")

训练集特征数量: 4
测试集特征数量: 4
特征名称: ['Age' 'Pclass' 'Sex=female' 'Sex=male']


In [225]:
# 决策树API当中，如果没有指定max_depth那么会根据信息熵的条件直到最终结束。
# 这⾥我们可以指定树的深度来进⾏限制树的⼤⼩

# 4.机器学习（决策树）
# max_depth=5：这个参数设置决策树的最大深度为 5。限制树的深度有助于防止过拟合，确保模型的泛化能力。
estimator = DecisionTreeClassifier(criterion="entropy", max_depth=5)
estimator.fit(x_train, y_train)

# 5.模型评估
estimator_score = estimator.score(x_test,y_test)
print(f"模型在测试集上的得分: {estimator_score}")

estimator.predict(x_test)

模型在测试集上的得分: 0.7805907172995781


array([0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0.,
       0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 0.

In [226]:
# 保存树的结构到dot文件中
export_graphviz(estimator, out_file="./data/tree.dot", feature_names=feature_names)


In [232]:
# 导入必要的包
import pandas as pd
import numpy as ny
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
# export_graphviz 函数是 sklearn.tree 模块中的一个工具，用于将决策树模型导出为 Graphviz 格式的 DOT 文件。
# 这个格式可以被 Graphviz 软件可视化，便于分析和理解决策树的结构。
from sklearn.tree import DecisionTreeClassifier, export_graphviz

# 1.获取数据
titanic = pd.read_csv("./data/Titanic.txt",encoding='ISO-8859-1')
# titanic.head()

# 检查 Pclass 列
print(titanic['Pclass'].unique())

# 将 Pclass 转为字符串类型（如有必要）
# 数据类型问题：如果 Pclass 列是字符串类型，而你在处理时没有正确转换，可能导致它被视为单一类别。
titanic['Pclass'] = titanic['Pclass'].astype(str)

# 2.基本数据处理
titanic = titanic.dropna(subset=["Survived"])
#  2.1 确定特征值，目标值
x = titanic[["Pclass","Age","Sex"]]
y = titanic["Survived"]

#  2.2 缺失值处理
# inplace=True：这个参数表示在原地修改数据框，即直接在 x 数据框中替换缺失值，而不是返回一个新的数据框。
x['Age'].fillna(x['Age'].mean(), inplace=True)

#  2.3 分割数据
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=22)

# 3.特征工程
# 对于x转换成字典数据x.to_dict(orient="records")
# 这个方法将数据框 x 转换为字典格式，每一行转换为一个字典，字典的键是列名，值是对应的列值。
# 例如，若 x 数据框的一行包含 pclass、age 和 sex 三个特征，则转换后可能得到 {"pclass": "1st", "age": 29.00, "sex": "female"} 这样的字典。
# orient="records" 参数表示每一行转换为一个字典，并将所有这些字典放入一个列表中。
# 例如：[{"pclass": "1st", "age": 29.00, "sex": "female"}, {}]

# DictVectorizer 将字典格式的数据转换为数组格式。
# sparse=False 参数表示输出为一个密集数组（即常规的 NumPy 数组），而不是稀疏矩阵。
transfer = DictVectorizer(sparse=False)

# 在训练集上拟合并转换
x_train = transfer.fit_transform(x_train.to_dict(orient="records"))
# 在测试集上仅转换
x_test = transfer.transform(x_test.to_dict(orient="records"))

print(f"训练集特征数量: {x_train.shape[1]}")
print(f"测试集特征数量: {x_test.shape[1]}")

# 获取特征名称
feature_names = transfer.get_feature_names_out()
print(f"特征名称: {feature_names}")

# 决策树API当中，如果没有指定max_depth那么会根据信息熵的条件直到最终结束。
# 这⾥我们可以指定树的深度来进⾏限制树的⼤⼩

# 4.机器学习（决策树）
# max_depth=5：这个参数设置决策树的最大深度为 5。限制树的深度有助于防止过拟合，确保模型的泛化能力。
estimator = DecisionTreeClassifier(criterion="entropy", max_depth=5)
estimator.fit(x_train, y_train)

# 5.模型评估
estimator_score = estimator.score(x_test,y_test)
print(f"模型在测试集上的得分: {estimator_score}")

estimator.predict(x_test)

# 保存树的结构到dot文件中
# ['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', '女性', '男性']
export_graphviz(estimator, out_file="./data/tree.dot", feature_names=feature_names)


[3 1 2]
训练集特征数量: 6
测试集特征数量: 6
特征名称: ['Age' 'Pclass=1' 'Pclass=2' 'Pclass=3' 'Sex=female' 'Sex=male']
模型在测试集上的得分: 0.7805907172995781


C:\Users\ZSJ\AppData\Local\Temp\ipykernel_10124\26815099.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['Age'].fillna(x['Age'].mean(), inplace=True)
C:\Users\ZSJ\AppData\Local\Temp\ipykernel_10124\26815099.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Age'].fillna(x['Age'].mean(), inplace=True)
